# Block 7 Übung 6: Visualisierung & Geovisualisierung von RKI-Grippedaten

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "tidyr", "lubridate", "ggplot2", "sf", "leaflet")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"


## RKI-Grippedaten einlesen

Quelle: Robert Koch-Institut: SurvStat@RKI 2.0, https://survstat.rki.de, Abfragedatum: 28.06.2021

Filterkriterien:
* Erkrankung: Influenza
* Zeitraum: unbeschränkt
* Ausgabe: Fallzahl vs. Meldejahr & Kreis

Nachbearbeitung:
* Pivotierung des Wide-Formats (Landkreise = Spalten) in ein besser handhabbares Long-Format

In [ ]:
# RKI-Grippedaten einlesen
rki.grippe.long <- read_delim(paste(base_url, "%C3%9Cbung/rki_grippe_long.csv", sep=""), delim = ";",
                                 col_types = cols(meldejahr = col_integer(), kreis_name = col_character(), fallzahl_influenza = col_double()))
head(rki.grippe.long)

## Aufgabe: Visualisieren Sie den Verlauf der Grippe-Infektionen über die Zeit insgesamt, sowie innerhalb einzelner Landkreise!

### Verlauf Gesamtfallzahl

### Verlauf einzelne Stadt-/Landkreise

Hinweise:
* Wählen Sie im zweiten Schritt Kreise aus, die unterschiedliche Bevölkerung haben (Großstadt vs. kleinere Stadt)
* Suchen sie hierzu mit der `grepl()`-Funktion nach entsprechenden Kreis-Namen

### Einschätzung: sind die Daten der verschiedenen Kreise ohne weiteres miteinander vergleichbar?

* der RKI-Datensatz enthält absolute Fallzahlen

## Bezugsgröße (Bevölkerung/Kreis) herunterladen

Quelle: Statistisches Bundesamt [Tabelle 12411-0015](https://www-genesis.destatis.de/genesis//online?operation=table&code=12411-0015&bypass=true&levelindex=1&levelid=1624946140391) (Bevölkerung: Kreise, Stichtag)

* Download im Flatfile-Format, keine Nachbearbeitung

In [ ]:
destatis.bevoelkerung.raw <- read_delim(paste(base_url, "%C3%9Cbung/12411-0015_flat.csv", sep=""), delim = ";", 
                                        col_types = cols(Statistik_Code = col_integer(), Statistik_Label = col_character(), Zeit_Code = col_character(), Zeit_Label = col_character(), Zeit = col_date(format = "%d.%m.%Y"), `1_Merkmal_Code` = col_character(), `1_Merkmal_Label` = col_character(), `1_Auspraegung_Code` = col_character(), `1_Auspraegung_Label` = col_character(), BEVSTD__Bevoelkerungsstand__Anzahl = col_character()),
                                        locale = locale(encoding = "UTF8"))

# Nachbearbeitung: fehlende Werte sind mit "-" angegeben, was gegen NA ausgetauscht werden muss
destatis.bevoelkerung.raw$BEVSTD__Bevoelkerungsstand__Anzahl[which(destatis.bevoelkerung.raw$BEVSTD__Bevoelkerungsstand__Anzahl == "-")] <- NA

# Jetzt kann die Spalte zu einer Zahl umgewandelt werden, da sie keine nichtnumerischen Zeichen mehr enthält
destatis.bevoelkerung.raw$BEVSTD__Bevoelkerungsstand__Anzahl <- as.integer(destatis.bevoelkerung.raw$BEVSTD__Bevoelkerungsstand__Anzahl)

# Relevante Spalten selektieren & mit besser handhabbaren Namen versehen
destatis.bevoelkerung.nice <- destatis.bevoelkerung.raw %>% 
    mutate(jahr = as.integer(year(Zeit)), ags_kreis_id = `1_Auspraegung_Code`, ags_kreis_name = `1_Auspraegung_Label`, bevoelkerung = BEVSTD__Bevoelkerungsstand__Anzahl) %>%
    select(jahr, ags_kreis_id, ags_kreis_name, bevoelkerung)

head(destatis.bevoelkerung.nice)

## Aufgabe: Prüfen Sie, ob ein Join zwischen den Landkreisen im Grippedatensatz des RKI und den Bevölkerungsdaten des Statistischen Bundesamts hergestellt werden kann!

Vorgehensweise:
* sichten sie die Spalten in den beiden Datensätzen: gibt es Spalten, die gemeinsam/identisch sein könnten?
* prüfen Sie möglicherweise nutzbare Spalten ggf. über einen Left Join der RKI- mit den Bevölkerungsdaten - gibt es für alle Zeilen eine Abbildung, oder bleiben Datensätze mit NA (Not Available) in den Spalten der Bevölkerungs-Tabelle?
* falls keine passende Spalte verfügbar ist: welche Spalte in den Bevölkerungsdaten könnte am besten genutzt werden, wenn sie in der RKI-Tabelle vorhanden wäre?

## Das Missing Link: Mappingtabelle zwischen den vom RKI verwendeten Stadt-/Landkreisnamen und den offiziellen IDs aus dem amtlichen Gemeindeschlüssel (AGS)

Auf Github wird unter https://github.com/noz-mhn-data/RKI-merge eine CSV-Datei mit einem Mapping zwischen den vom RKI verwendeten Stadt-/Landkreisen auf AGS-Schlüssel angeboten, diese wurde allerdings seit 2019 nicht mehr aktualisiert.

Ein Fork mit einer aktualisierten Version für den Kurs befindet sich unter https://github.com/ganslats/RKI-merge

In [ ]:
rki.kreise.map <- read_delim("https://raw.githubusercontent.com/ganslats/RKI-merge/master/Merging_Tabelle.CSV", delim = ";", skip = 1, 
                             col_names = c("kreis_name", "kreis_id"),
                             col_types = cols(kreis_name = col_character(), kreis_id = col_character()))
head(rki.kreise.map)

## Aufgabe: Ergänzen Sie die Kreis-IDs aus dem Mapping in den Grippe-Daten des RKI! 

## Aufgabe: Berechnen Sie die Inzidenz pro 100.000 Einwohner & plotten Sie die gleichen Stadt/Landkreise wie oben!

Hinweise:
* führen Sie hierzu die RKI-Daten per Join mit den Bevölkerungsdaten zusammen und ergänzen eine berechnete Spalte mit der `mutate()`-Funktion

## Vorbereitung Geovisualisierung: Umrisse der Landkreise einlesen

**Wichtiger Hinweis:** Wenn Sie an dieser Stelle eine Fehlermeldung erhalten, müssen Sie die ersten beiden Zeilen einkommentieren (# entfernen) und einmalig ausführen, um die Rohdaten von https://gadm.org/ herunterzuladen. Anschließend können die Zeilen wieder auskommentiert werden. Quellenangaben und Hintergrundinfos siehe [teil-6-demo-02-visualisieren-geo.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-6-demo-02-visualisieren-geo.ipynb)

Hinweis: es kann in Binder hier eine Fehlermeldung über eine fehlende Bibliothek "geojsonio" geben, die an dieser Stelle aber ignoriert werden kann.

In [ ]:
#download.file("https://biogeo.ucdavis.edu/data/gadm3.6/gpkg/gadm36_DEU_gpkg.zip", "gadm36_DEU_gpkg.zip")
#unzip("gadm36_DEU_gpkg.zip", "gadm36_DEU.gpkg", junkpaths = TRUE)
shapes.landkreise <- st_read("gadm36_DEU.gpkg", layer="gadm36_DEU_2")
head(shapes.landkreise)

## Aufgabe: Führen Sie die Influenza-Daten mit den Umrissdaten zusammen und erstellen Sie eine Geovisualisierung mit Leaflet oder ggmap!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) und dem Abschnitt 6, Demo 2 (Geovisualisierung) anwenden.

* zugehörige Notebooks: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb), [teil-6-demo-02-visualisieren-geo.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-6-demo-02-visualisieren-geo.ipynb)

Hinweise:
* die Kennzahlen aus der oben berechneten Tabelle `influenza.inzidenz` müssen per left_join in das Tibble mit den Umrissen übernommen werden, da sonst Landkreise ohne Fälle verschwinden würden
* da die Tabelle Daten mehrerer Jahre enthält, müssen sie zuvor auf ein von Ihnen ausgewähltes Jahr gefiltert werden (z.B. Grippewelle 2019), da die Werte der verschiedenen Jahre sonst "übereinander" geplottet werden würden

### Inzidenzen eines Bezugsjahrs auswählen und den Umrissdaten hinzufügen

### Leaflet-Karte mit den Inzidenzen anzeigen